<a href="https://colab.research.google.com/github/dan-a-iancu/airm/blob/master/OR_Scheduling/Operating_Room_Scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook implements the solution to the **Operating Room Scheduling** mini-case. It assumes you are familiar with the case and the model.

____
# Basic Setup

Import useful modules, read the data and store it in data frames, and set up some useful Python lists. You may want to expand this section and make sure you understand how the data is organized, and also read the last part where the Python lists are created, as these may be very useful when you build your model.

In [1]:
#@markdown We first import some useful modules. 

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# import numpy
import numpy as np
import urllib.request  # for file downloading

# Import pandas for data-frames
import pandas as pd
pd.options.display.max_rows = 15
pd.options.display.float_format = "{:,.2f}".format

from IPython.display import display

# Make sure Matplotlib runs inline, for nice figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import matplotlib.ticker as ticker 

# install Gurobi (our linear optimization solver)
!pip install -i https://pypi.gurobi.com gurobipy
from gurobipy import *

# some modules to create local directories for CBC (to avoid issues with solving multiple models)
import os
def new_local_directory(name):
    full_path = os.path.join(".", name)
    os.makedirs(full_path, exist_ok=True)
    return full_path

# install the latest version of seaborn for nicer graphics
#!pip install --prefix {sys.prefix} seaborn==0.11.0  &> /dev/null
import seaborn as sns

# Ignore useless some warnings
import warnings
warnings.simplefilter(action="ignore")

print("Completed successfully!")

Looking in indexes: https://pypi.gurobi.com
     |████████████████████████████████| 11.1MB 410kB/s 
Completed successfully!


## Load the case data into Pandas data frames

We first download an Excel file with all the data from Github.

In [2]:
#@markdown Download the entire data as an Excel file from Github

url_Excel = 'https://github.com/dan-a-iancu/airm/blob/master/OR_Scheduling/OR_Scheduling_data.xlsx?raw=true'
local_file = "OR_Scheduling_Data.xlsx"   # name of local file where you want to store the downloaded file
urllib.request.urlretrieve(url_Excel, local_file)    # download from website and save it locally

('OR_Scheduling_Data.xlsx', <http.client.HTTPMessage at 0x7f80677c45d0>)

Read in and store the data in suitable dataframes.

In [3]:
#@markdown Create dataframes based on the Excel file

# data on hours requested (Table 1)
hours_df = pd.read_excel("OR_Scheduling_Data.xlsx", sheet_name = "Requests", index_col=0)
display(hours_df)

# data on available surgical teams (Table 2); "index_col=0" just assigns the first column as the row names of the data frame
teams_df = pd.read_excel("OR_Scheduling_Data.xlsx", sheet_name = "Teams", index_col=0)
display(teams_df)

,Opthalmology,Gynecology,Oral_Surgery,Otolaryngology,General_Surgery
Hours,20,39,14,17,63


,Opthalmology,Gynecology,Oral_Surgery,Otolaryngology,General_Surgery
Mon,2,2,0,3,4
Tue,4,4,2,1,4
Wed,2,4,0,1,4
Thu,4,2,2,2,4
Fri,3,2,0,2,4


## Create Python lists and store any other relevant problem parameters

__NOTE__: Make sure you understand what the __lists__ created here are! These will be very helpful when creating the model.

In [4]:
#@markdown Create useful lists with the departments and the days of the week
# calculate the list of departments
departments = list( hours_df.columns )
print("The list of departments:")
print(departments)

# calculate the list of days
days = list( teams_df.index )
print("The list of days:")
print(days)

The list of departments:
['Opthalmology', 'Gynecology', 'Oral_Surgery', 'Otolaryngology', 'General_Surgery']
The list of days:
['Mon', 'Tue', 'Wed', 'Thu', 'Fri']


In [5]:
#@markdown Also create a separate list with the names of the blocks within a day
blocks_of_day = ['8am-11am', '11am-2pm', '2pm-5pm', '5pm-8pm']
print("The blocks in a day:")
print(blocks_of_day)

# the length of time for a block
block_length = 3

The blocks in a day:
['8am-11am', '11am-2pm', '2pm-5pm', '5pm-8pm']


<font color=red>**IMPORTANT HINT.**</font> When defining your **decision variables** in this model, you may want to consider *matrices* (or grids) of decision variables instead of a simple list like we've created before. Specifically, you may want to add a decision variable for every pair of elements where the first element is from one list and the second element is from another list. 

When the decision variables are organized in a grid, it's significantly easier to use the ``addVars`` function to add all of the decision variables to the model at once. The function ``addVars`` can take as arguments several lists, e.g., `addVars(list1, list2, list3,...)`, in which case it will return one decision variable for every possible tuple of values. These decision variables will be stored as a dictionary, with the dictionary keys corresponding to the tuple of values. 

For instance, suppose `list1 = ["Paris", "Budapest", "Beijing"]` and `list2= [1,2]`. Then `myDecisions = addVars(list1,list2)` would return a dictionary containing one decision for every pair, i.e., ("Paris",1), ("Paris",2), ("Budapest",1), ("Budapest",2), etc. To access the decision corresponding to the pair ("Paris",1), we can simply use `myDecisions[("Paris",1)]` or `myDecisions["Paris",1]`.

_____
# **Q1**

##  Create an empty model

In [6]:
# Gurobi model
OR_Model = Model("Operating Room Scheduling Model")

Restricted license - for non-production use only - expires 2022-01-13


## Define the Decision Variables
In this problem, we would like to create a decision variable for every triple of (day, block, department). This is done easily using Gurobi's ``addVars`` method.

In [7]:
# decision variables for how much to ship from each factory to each customization center
block_assigned = OR_Model.addVars( days, blocks_of_day, departments , name = "Assign", vtype=GRB.BINARY)

## Calculate and add the objective function

In [8]:
# Calculate the percentage coverage for each department and store it in a dictionary
coverage = {}
for dept in departments:
  coverage[dept] = quicksum(block_assigned[day,block,dept] \
                            for day in days for block in blocks_of_day) * \
                            block_length / hours_df[dept]["Hours"]

total_coverage = quicksum( coverage[dept] for dept in departments )

# objective: maximize the total coverage
OR_Model.setObjective(total_coverage, GRB.MAXIMIZE)

## Add All Constraints

In [9]:
# each block is assigned at most once
for day in days:
    for block in blocks_of_day:
        OR_Model.addConstr( quicksum(block_assigned[day,block,dept] for dept in departments) <= 1 )

# the total blocks assigned to any department in any day do not exceed the available surgical teams
for day in days:
    for dept in departments:
        OR_Model.addConstr( quicksum(block_assigned[day,block,dept] \
                                     for block in blocks_of_day) <= \
                           teams_df[dept][day])

## Inspect the model


You can actually write the model out, which is useful for inspection. In this case the LP file may become large, so the code is commented out below. Feel free to un-comment the second line to see the output.

In [10]:
#@markdown Feel free to uncomment the next lines if you want to see the model printed
# OR_Model.write("OR_Model.lp")    # write the model to a file with extension ".lp"
# f = open("OR_Model.lp", 'r')    # open a file handle
# print( f.read() )           # read the contents and print them
# f.close()              # close the file handle

## Solve the model

In [11]:
#@markdown Select whether to run the [Gurobi](https://www.gurobi.com/) optimization algorithms silently (no output details)
run_silently = False #@param {type:"boolean"}

if run_silently:
    OR_Model.setParam('OutputFlag',0)
else:
    OR_Model.setParam('OutputFlag',1)

OR_Model.optimize()
print('\nSolved the optimization problem...')

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 45 rows, 100 columns and 200 nonzeros
Model fingerprint: 0x77132d15
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.6346154
Presolve removed 38 rows and 88 columns
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 24 nonzeros
Found heuristic solution: objective 3.4223012
Variable types: 0 continuous, 12 integer (12 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 2 (of 2 available processors)

Solution count 2: 3.4223 2.63462 

Optimal solution found (tolerance 1.00e-04)
Best 

## Print the optimal coverage and optimal schedule

In [12]:
#@title Define a function to print the schedule and coverage
def print_schedule_and_coverage(model, block_assigned, coverage):
    #print("="*60)
    #print("Schedule:\n")
    # for day in days:
    #     print("\n{:<15s} | {:<15s} | {:<15s} | {:<15s} | {:<15s} | {:<15s}".\
    #           format(day,*departments)) 
    #     for block in blocks_of_day:
    #         print("{:<15s} | {:<15d} | {:<15d} | {:<15d} | {:<15d} | {:<15d}".\
    #               format(block,*[np.int(block_assigned[day,block,dept].X) for dept in departments]))

    # print the schedule as a pandas dataframe with multi-index
    pd.options.display.max_rows = 25
    iterables = [days, blocks_of_day]
    df_schedule = pd.DataFrame(dict( [ (dept, [np.int(block_assigned[day,block,dept].X) for day in days \
                                               for block in blocks_of_day]) for dept in departments]),\
                               index=pd.MultiIndex.from_product(iterables, names=["Day", "Block"]))
    display(df_schedule)

    print("\nCoverage by department:")
    print("="*100)
    print("{:<15s} : {:<15s} : {:<15s}".format("Department","Claimed coverage",\
                                               "Effective coverage"))
    for dept in departments:
        claimed_cov_perc = 100*coverage[dept].getValue()
        print("{:<15s} : {:<15.2f}% : {:<15.2f}%".\
              format(dept,claimed_cov_perc,min(100,claimed_cov_perc)))

    print("\nTotal coverage : {:.2f}%\n".format(100*model.objVal))

#@markdown Use the function to print the results
print_schedule_and_coverage(OR_Model, block_assigned, coverage)



Opthalmology  Gynecology  ...  Otolaryngology  General_Surgery
Day Block                               ...                                 
Mon 8am-11am             0           0  ...               1                0
    11am-2pm             1           0  ...               0                0
    2pm-5pm              0           0  ...               1                0
    5pm-8pm              0           0  ...               1                0
Tue 8am-11am             0           0  ...               0                0
    11am-2pm             0           0  ...               0                0
    2pm-5pm              0           0  ...               1                0
    5pm-8pm              1           0  ...               0                0
Wed 8am-11am             1           0  ...               0                0
    11am-2pm             1           0  ...               0                0
    2pm-5pm              0           0  ...               1                0
    5pm-8pm              0           1  ...               0                0
Thu 8am-11am             0           0  ...               0                0
    11am-2pm             0           0  ...               0                0
    2pm-5pm              0           0  ...               1                0
    5pm-8pm              0           0  ...               1                0
Fri 8am-11am             0           0  ...               1                0
    11am-2pm             0           0  ...               1                0
    2pm-5pm              1           0  ...               0                0
    5pm-8pm              1           0  ...               0                0

[20 rows x 5 columns]


Coverage by department:
Department      : Claimed coverage : Effective coverage
Opthalmology    : 90.00          % : 90.00          %
Gynecology      : 7.69           % : 7.69           %
Oral_Surgery    : 85.71          % : 85.71          %
Otolaryngology  : 158.82         % : 100.00         %
General_Surgery : 0.00           % : 0.00           %

Total coverage : 342.23%



Store the optimal coverage in **Q1**.

In [13]:
coverage_Q1 = OR_Model.objVal  # optimal coverage Q1

## Create a few utility functions that will help subsequently

To help with subsequent parts of the problem, we also add all the steps above inside a **function** that returns a generic model like the one we created in **Q1**, together with all the decision variables.

In [14]:
#@title A function that generates a model like the one in **Q1**
def create_model_like_in_Q1():
    #@markdown Empty Gurobi model
    OR_Model = Model("Operating Room Scheduling Model")

    #@markdown Decision vars
    # decision variables for how much to ship from each factory to each customization center
    block_assigned = OR_Model.addVars( days, blocks_of_day, departments , name = "Assign", vtype=GRB.BINARY)

    #@markdown Objective
    # Calculate the percentage coverage for each department and store it in a dictionary
    coverage = {}
    for dept in departments:
      coverage[dept] = quicksum(block_assigned[day,block,dept] \
                                for day in days for block in blocks_of_day) * \
                                block_length / hours_df[dept]["Hours"]

    total_coverage = quicksum( coverage[dept] for dept in departments )

    # objective: maximize the total coverage
    OR_Model.setObjective(total_coverage, GRB.MAXIMIZE)

    #@markdown Constraints
    # each block is assigned at most once
    for day in days:
        for block in blocks_of_day:
            OR_Model.addConstr( quicksum(block_assigned[day,block,dept] for dept in departments) <= 1 )

    # the total blocks assigned to any department in any day do not exceed the available surgical teams
    for day in days:
        for dept in departments:
            OR_Model.addConstr( quicksum(block_assigned[day,block,dept] \
                                        for block in blocks_of_day) <= \
                              teams_df[dept][day])

    # return the model, the decision variables, the individual coverage calculations
    return OR_Model,  block_assigned, coverage,  total_coverage

______
# **Q2**
Before running this section, make sure you have run all the previous sections of the Colab file.

Re-recreate an identical model to the one from **Q1** and store the expression for model, the decision variables and the objective.

In [15]:
#@markdown Create a model like the one in Q1 using the function 
OR_Model,  block_assigned, coverage,  total_coverage = \
       create_model_like_in_Q1()

Add a new set of decision variables for the actual coverage, which can never exceed 100\%.

In [16]:
#@markdown Add a new set of decisions for the actual coverage
actual_coverage = OR_Model.addVars( departments )

Calculate the total coverage objective using the new decision variables.

In [17]:
#@markdown Adjust the objective to use the actual coverage
OR_Model.setObjective( quicksum(actual_coverage[dept] for dept in departments),\
                      GRB.MAXIMIZE)

Add constraints that the actual coverage variables are at most equal to the calculated coverage and are at most 1, for every department.

In [18]:
#@markdown Add constraints on the **actual coverage** variables
for dept in departments:
    OR_Model.addConstr( actual_coverage[dept] <= coverage[dept], name='Actual_leq_calculated' )
    OR_Model.addConstr( actual_coverage[dept] <= 1.0, name='Actual_leq_1' )

Set the objective in the new model to maximize the total coverage calculated using the **actual coverage** variables.

In [19]:
#@title Calculate objective using the new(actual coverage) variables
OR_Model.setObjective( quicksum( actual_coverage[dept] for dept in departments ),\
                      GRB.MAXIMIZE )

#@markdown Select whether to run the [Gurobi](https://www.gurobi.com/) optimization algorithms silently (no output details)
run_silently = True #@param {type:"boolean"}

if run_silently:
    OR_Model.setParam('OutputFlag',0)
else:
    OR_Model.setParam('OutputFlag',1)

OR_Model.optimize()
print('\nSolved the optimization problem...')


Solved the optimization problem...


Print the optimal solution.

In [20]:
#@markdown Use the function to print the results
print_schedule_and_coverage(OR_Model, block_assigned, coverage)

Opthalmology  Gynecology  ...  Otolaryngology  General_Surgery
Day Block                               ...                                 
Mon 8am-11am             1           0  ...               0                0
    11am-2pm             0           0  ...               1                0
    2pm-5pm              0           0  ...               1                0
    5pm-8pm              0           0  ...               1                0
Tue 8am-11am             0           1  ...               0                0
    11am-2pm             0           0  ...               0                0
    2pm-5pm              0           0  ...               0                0
    5pm-8pm              1           0  ...               0                0
Wed 8am-11am             1           0  ...               0                0
    11am-2pm             0           1  ...               0                0
    2pm-5pm              1           0  ...               0                0
    5pm-8pm              0           1  ...               0                0
Thu 8am-11am             0           0  ...               1                0
    11am-2pm             0           0  ...               0                0
    2pm-5pm              1           0  ...               0                0
    5pm-8pm              0           0  ...               0                0
Fri 8am-11am             0           0  ...               1                0
    11am-2pm             1           0  ...               0                0
    2pm-5pm              0           0  ...               1                0
    5pm-8pm              1           0  ...               0                0

[20 rows x 5 columns]


Coverage by department:
Department      : Claimed coverage : Effective coverage
Opthalmology    : 105.00         % : 100.00         %
Gynecology      : 23.08          % : 23.08          %
Oral_Surgery    : 85.71          % : 85.71          %
Otolaryngology  : 105.88         % : 100.00         %
General_Surgery : 0.00           % : 0.00           %

Total coverage : 308.79%



______
# **Q3**
Before running this section, make sure you have run all the previous sections of the Colab file.

Re-recreate an identical model to the one from **Q1** and store the expression for model, the decision variables and the objective.

In [21]:
#@markdown Create a model like the one in Q1 using the function 
OR_Model,  block_assigned, coverage,  total_coverage = \
       create_model_like_in_Q1()

Add a constraint to ensure that **if Gynecology does not get Monday 8-11am, they do get the Monday 11am-2pm slot.**

Let $(Dep,D,i)$ capture a binary variable for whether department $Dep$ is assigned on day $D \in \{M,Tu,W,Th,F\}$ block number $i$, where block 1 is the 11am-2pm slot, block 2 is 11am-2pm slot, etc. Then, the constraint we want to add is:
>  1 - `(Gy,M,1)` $\leq$ `(Gy,M,2)`

In [22]:
#@markdown Add suitable constraint
OR_Model.addConstr( 1 - block_assigned["Mon","8am-11am","Gynecology"] <=\
                   block_assigned["Mon","11am-2pm","Gynecology"] )

#@markdown Select whether to run the [Gurobi](https://www.gurobi.com/) optimization algorithms silently (no output details)
run_silently = True #@param {type:"boolean"}

if run_silently:
    OR_Model.setParam('OutputFlag',0)
else:
    OR_Model.setParam('OutputFlag',1)

OR_Model.optimize()
print('\nSolved the optimization problem...')


Solved the optimization problem...


Print the optimal solution.

In [23]:
#@markdown Use the function to print the results
print_schedule_and_coverage(OR_Model, block_assigned, coverage)

Opthalmology  Gynecology  ...  Otolaryngology  General_Surgery
Day Block                               ...                                 
Mon 8am-11am             0           0  ...               1                0
    11am-2pm             0           1  ...               0                0
    2pm-5pm              0           0  ...               1                0
    5pm-8pm              0           0  ...               1                0
Tue 8am-11am             0           0  ...               0                0
    11am-2pm             0           0  ...               0                0
    2pm-5pm              0           0  ...               1                0
    5pm-8pm              1           0  ...               0                0
Wed 8am-11am             1           0  ...               0                0
    11am-2pm             1           0  ...               0                0
    2pm-5pm              0           0  ...               1                0
    5pm-8pm              0           1  ...               0                0
Thu 8am-11am             0           0  ...               0                0
    11am-2pm             0           0  ...               0                0
    2pm-5pm              0           0  ...               1                0
    5pm-8pm              0           0  ...               1                0
Fri 8am-11am             1           0  ...               0                0
    11am-2pm             0           0  ...               1                0
    2pm-5pm              1           0  ...               0                0
    5pm-8pm              0           0  ...               1                0

[20 rows x 5 columns]


Coverage by department:
Department      : Claimed coverage : Effective coverage
Opthalmology    : 75.00          % : 75.00          %
Gynecology      : 15.38          % : 15.38          %
Oral_Surgery    : 85.71          % : 85.71          %
Otolaryngology  : 158.82         % : 100.00         %
General_Surgery : 0.00           % : 0.00           %

Total coverage : 334.92%



______
# **Q4**
Before running this section, make sure you have run all the previous sections of the Colab file.

Here, the manager wants to ensure that **if any morning slot (i.e., 8am-11am) is assigned to Ophthalmology during the week, then at least one morning slot is assigned to Gynecology**. Let $(Dep,D,i)$ capture a binary variable that denotes whether department $Dep$ is assigned on day $D \in \{M,Tu,W,Th,F\}$ block number $i$, where block 1 is the 11am-2pm slot, block 2 is 11am-2pm slot, etc. 

#### **OPTION 1.** We can add all the constraints of the form:
> $(Op,M,1) \leq \sum_{D \in \{M,Tu,W,Th,F\}} (Gy,D,1)$<br>
> $(Op,Tu,1) \leq \sum_{D \in \{M,Tu,W,Th,F\}} (Gy,D,1)$<br>
> ...<br>
> $(Op,F,1) \leq \sum_{D \in \{M,Tu,W,Th,F\}} (Gy,D,1)$.

Re-recreate an identical model to the one from **Q1** and store the expression for model, the decision variables and the objective.

In [35]:
#@markdown Create a model like the one in Q1 using the function 
OR_Model,  block_assigned, coverage,  total_coverage = \
       create_model_like_in_Q1()

In [36]:
#@markdown **OPTION 1**
for day in days:
    OR_Model.addConstr( block_assigned[day,"8am-11am","Opthalmology"] <=\
                      quicksum(block_assigned[d,"8am-11am","Gynecology"] for d in days) )

#@markdown Select whether to run the [Gurobi](https://www.gurobi.com/) optimization algorithms silently (no output details)
run_silently = False #@param {type:"boolean"}

if run_silently:
    OR_Model.setParam('OutputFlag',0)
else:
    OR_Model.setParam('OutputFlag',1)

OR_Model.optimize()
print('\nSolved the optimization problem...')

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 50 rows, 100 columns and 230 nonzeros
Model fingerprint: 0xf000eb3c
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.6346154
Presolve removed 13 rows and 47 columns
Presolve time: 0.00s
Presolved: 37 rows, 53 columns, 126 nonzeros
Variable types: 0 continuous, 53 integer (53 binary)

Root relaxation: objective 3.422301e+00, 14 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       3.4223012    3.42230  0.00%  

Print the optimal solution.

In [37]:
#@markdown Use the function to print the results
print_schedule_and_coverage(OR_Model, block_assigned, coverage)

Opthalmology  Gynecology  ...  Otolaryngology  General_Surgery
Day Block                               ...                                 
Mon 8am-11am             0           0  ...               1                0
    11am-2pm             1           0  ...               0                0
    2pm-5pm              0           0  ...               1                0
    5pm-8pm              0           0  ...               1                0
Tue 8am-11am             0           0  ...               1                0
    11am-2pm             1           0  ...               0                0
    2pm-5pm              0           0  ...               0                0
    5pm-8pm              0           0  ...               0                0
Wed 8am-11am             0           0  ...               1                0
    11am-2pm             1           0  ...               0                0
    2pm-5pm              0           1  ...               0                0
    5pm-8pm              1           0  ...               0                0
Thu 8am-11am             0           0  ...               1                0
    11am-2pm             0           0  ...               1                0
    2pm-5pm              0           0  ...               0                0
    5pm-8pm              0           0  ...               0                0
Fri 8am-11am             0           0  ...               1                0
    11am-2pm             1           0  ...               0                0
    2pm-5pm              1           0  ...               0                0
    5pm-8pm              0           0  ...               1                0

[20 rows x 5 columns]


Coverage by department:
Department      : Claimed coverage : Effective coverage
Opthalmology    : 90.00          % : 90.00          %
Gynecology      : 7.69           % : 7.69           %
Oral_Surgery    : 85.71          % : 85.71          %
Otolaryngology  : 158.82         % : 100.00         %
General_Surgery : 0.00           % : 0.00           %

Total coverage : 342.23%



#### **OPTION 2.** We can add a single constraint of the form:
> $\frac{\sum_{D \in \{M,Tu,W,Th,F\}} (Op,D,1)}{5} \leq \sum_{D \in \{M,Tu,W,Th,F\}} (Gy,D,1)$.

Re-recreate an identical model to the one from **Q1** and store the expression for model, the decision variables and the objective.

In [38]:
#@markdown Create a model like the one in Q1 using the function 
OR_Model,  block_assigned, coverage,  total_coverage = \
       create_model_like_in_Q1()

In [39]:
#@markdown **OPTION 2**
OR_Model.addConstr( quicksum(block_assigned[d,"8am-11am","Opthalmology"] \
                             for d in days)/len(days) <= \
                   quicksum(block_assigned[d,"8am-11am","Gynecology"] for d in days) )

#@markdown Select whether to run the [Gurobi](https://www.gurobi.com/) optimization algorithms silently (no output details)
run_silently = False #@param {type:"boolean"}

if run_silently:
    OR_Model.setParam('OutputFlag',0)
else:
    OR_Model.setParam('OutputFlag',1)

OR_Model.optimize()
print('\nSolved the optimization problem...')

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 46 rows, 100 columns and 210 nonzeros
Model fingerprint: 0x7bdade53
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [5e-02, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.6346154
Presolve removed 13 rows and 47 columns
Presolve time: 0.00s
Presolved: 33 rows, 53 columns, 111 nonzeros
Variable types: 0 continuous, 53 integer (53 binary)

Root relaxation: objective 3.422301e+00, 14 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.42230    0    5    2.63462    3.42230  29.9%  

Print the optimal solution.

In [40]:
#@markdown Use the function to print the results
print_schedule_and_coverage(OR_Model, block_assigned, coverage)

Opthalmology  Gynecology  ...  Otolaryngology  General_Surgery
Day Block                               ...                                 
Mon 8am-11am             0           0  ...               1                0
    11am-2pm             1           0  ...               0                0
    2pm-5pm              0           0  ...               1                0
    5pm-8pm              0           0  ...               1                0
Tue 8am-11am             0           0  ...               1                0
    11am-2pm             1           0  ...               0                0
    2pm-5pm              0           0  ...               0                0
    5pm-8pm              0           0  ...               0                0
Wed 8am-11am             0           0  ...               1                0
    11am-2pm             1           0  ...               0                0
    2pm-5pm              0           1  ...               0                0
    5pm-8pm              1           0  ...               0                0
Thu 8am-11am             0           0  ...               1                0
    11am-2pm             0           0  ...               1                0
    2pm-5pm              0           0  ...               0                0
    5pm-8pm              0           0  ...               0                0
Fri 8am-11am             0           0  ...               1                0
    11am-2pm             0           0  ...               1                0
    2pm-5pm              1           0  ...               0                0
    5pm-8pm              1           0  ...               0                0

[20 rows x 5 columns]


Coverage by department:
Department      : Claimed coverage : Effective coverage
Opthalmology    : 90.00          % : 90.00          %
Gynecology      : 7.69           % : 7.69           %
Oral_Surgery    : 85.71          % : 85.71          %
Otolaryngology  : 158.82         % : 100.00         %
General_Surgery : 0.00           % : 0.00           %

Total coverage : 342.23%



______
# **Q5**
Before running this section, make sure you have run all the previous sections of the Colab file.

Here, the manager wants to ensure that **General Surgery at least 2 consecutive blocks on Monday**. 

Let $(Dep,D,i)$ capture the same binary variables as in **Q3,4**. We add three new variables denoted by $Z_{12}, Z_{23}, Z_{34}$, with the meaning that $Z_{ij}=1$ if General Surgery is assigned both blocks $i$ and $j$ on Monday. And then we add the constraints:
> $Z_{12} + Z_{23} + Z_{34} \geq 1$ <br>
> $Z_{ij} \leq (\mbox{Ge},M,i), \, Z_{ij} \leq (\mbox{Ge},M,j)$, for all $(ij) \in \{(12),(23),(34)\}$.

In [30]:
# create a list with identifiers for the relevant pairs
pairs_consecutive_blocks = [ (blocks_of_day[i],blocks_of_day[i+1]) \
                            for i in range(len(blocks_of_day)-1) ]
print(pairs_consecutive_blocks)

[('8am-11am', '11am-2pm'), ('11am-2pm', '2pm-5pm'), ('2pm-5pm', '5pm-8pm')]


Re-recreate an identical model to the one from **Q1** and store the expression for model, the decision variables and the objective.

In [31]:
#@markdown Create a model like the one in Q1 using the function 
OR_Model,  block_assigned, coverage,  total_coverage = \
       create_model_like_in_Q1()

In [32]:
#@markdown Add the new decision variables

consecutive_block_assigned = OR_Model.addVars(pairs_consecutive_blocks)

In [33]:
#@markdown Add constraints ensuring they get at least two consecutive blocks
OR_Model.addConstr( quicksum(consecutive_block_assigned[p] \
                             for p in pairs_consecutive_blocks) >= 1)

#@markdown Add constraints to ensure that if the model claims to have assigned two consecutive blocks, it actually does it!
for p in pairs_consecutive_blocks:
    b1 = p[0]  # the first block in the pair p
    b2 = p[1]  # the second block in the pair p
    OR_Model.addConstr( consecutive_block_assigned[p] <= \
                       block_assigned["Mon",b1,"General_Surgery"] )
    OR_Model.addConstr( consecutive_block_assigned[p] <= \
                       block_assigned["Mon",b2,"General_Surgery"] )


#@markdown Select whether to run the [Gurobi](https://www.gurobi.com/) optimization algorithms silently (no output details)
run_silently = False #@param {type:"boolean"}

if run_silently:
    OR_Model.setParam('OutputFlag',0)
else:
    OR_Model.setParam('OutputFlag',1)

OR_Model.optimize()
print('\nSolved the optimization problem...')

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 52 rows, 103 columns and 215 nonzeros
Model fingerprint: 0xfbd52118
Variable types: 3 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.5760073
Presolve removed 43 rows and 92 columns
Presolve time: 0.00s
Presolved: 9 rows, 11 columns, 27 nonzeros
Found heuristic solution: objective 3.1910687
Variable types: 0 continuous, 11 integer (11 binary)

Root relaxation: cutoff, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0         

Print the optimal solution.

In [34]:
#@markdown Use the function to print the results
print_schedule_and_coverage(OR_Model, block_assigned, coverage)

Opthalmology  Gynecology  ...  Otolaryngology  General_Surgery
Day Block                               ...                                 
Mon 8am-11am             0           0  ...               1                0
    11am-2pm             0           0  ...               1                0
    2pm-5pm              0           0  ...               0                1
    5pm-8pm              0           0  ...               0                1
Tue 8am-11am             0           0  ...               0                0
    11am-2pm             0           0  ...               0                0
    2pm-5pm              0           0  ...               1                0
    5pm-8pm              1           0  ...               0                0
Wed 8am-11am             1           0  ...               0                0
    11am-2pm             1           0  ...               0                0
    2pm-5pm              0           0  ...               1                0
    5pm-8pm              0           1  ...               0                0
Thu 8am-11am             0           0  ...               0                0
    11am-2pm             0           0  ...               0                0
    2pm-5pm              0           0  ...               1                0
    5pm-8pm              0           0  ...               1                0
Fri 8am-11am             1           0  ...               0                0
    11am-2pm             0           0  ...               1                0
    2pm-5pm              1           0  ...               0                0
    5pm-8pm              0           0  ...               1                0

[20 rows x 5 columns]


Coverage by department:
Department      : Claimed coverage : Effective coverage
Opthalmology    : 75.00          % : 75.00          %
Gynecology      : 7.69           % : 7.69           %
Oral_Surgery    : 85.71          % : 85.71          %
Otolaryngology  : 141.18         % : 100.00         %
General_Surgery : 9.52           % : 9.52           %

Total coverage : 319.11%

